In [48]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.signal as signal
import scipy.stats as stats
import plotly.io as pio
from IPython.display import display, HTML
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.stattools import adfuller

pio.renderers.default = "notebook_connected"

def show_plotly(fig, include_plotlyjs="inline"):
    """Helper do wyświetlania wykresów Plotly."""
    try:
        fig.show()
    except Exception:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        display(HTML(html))


In [50]:
data_btcusdt_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./btcusd_day.csv"),
    usecols=["datetime", "open", "high", "low", "close"],
    parse_dates=["datetime"],
)
# Ustawiam odpowiednio indeks
data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
data_btcusdt_frame_daily["datetime"] = pd.to_datetime(
    data_btcusdt_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
data_btcusdt_frame_daily = data_btcusdt_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
data_btcusdt_frame_daily = (
    data_btcusdt_frame_daily.set_index("datetime").sort_index().astype("float64")
)

print(data_btcusdt_frame_daily.head(10))

               open     high      low    close
datetime                                      
2010-07-17  0.04951  0.04951  0.04951  0.04951
2010-07-18  0.08584  0.08584  0.08584  0.08584
2010-07-19  0.08080  0.08080  0.08080  0.08080
2010-07-20  0.07474  0.07474  0.07474  0.07474
2010-07-21  0.07921  0.07921  0.07921  0.07921
2010-07-22  0.05050  0.05050  0.05050  0.05050
2010-07-23  0.06262  0.06262  0.06262  0.06262
2010-07-24  0.05454  0.05454  0.05454  0.05454
2010-07-25  0.05050  0.05050  0.05050  0.05050
2010-07-26  0.05600  0.05600  0.05600  0.05600


In [51]:
strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
    Path("./equity_daily_normalized.csv"),
    usecols=["datetime", "equity"],
    parse_dates=["datetime"],
)
# strategy_equity_frame_daily: pd.DataFrame = pd.read_csv(
#     Path("./equity_daily_not_normalized.csv"),
#     usecols=["datetime", "equity"],
#     parse_dates=["datetime"],
# )

# Ustawiam odpowiednio indeks
strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")

# konwersja kolumny 'Data' na datetime, bo bez tego są błedne daty
strategy_equity_frame_daily["datetime"] = pd.to_datetime(
    strategy_equity_frame_daily["datetime"], errors="coerce"
)

# usuwam ewentualne błędne wiersze bez dat
strategy_equity_frame_daily.dropna(subset=["datetime"])

# indeks i sortuje
strategy_equity_frame_daily = (
    strategy_equity_frame_daily.set_index("datetime").sort_index().astype("float64")
)

strategy_equity_frame_daily.head(10)

,equity
datetime,
2022-01-01 00:00:00+00:00,1.000000
2022-01-02 00:00:00+00:00,1.000000
2022-01-03 00:00:00+00:00,1.000000
2022-01-04 00:00:00+00:00,0.997370
2022-01-05 00:00:00+00:00,1.050057
2022-01-06 00:00:00+00:00,1.057094
2022-01-07 00:00:00+00:00,1.090937
2022-01-08 00:00:00+00:00,1.088179
2022-01-09 00:00:00+00:00,1.084254


In [ ]:
# Wybór danych
# 1. Wybór danych (zgodnie z wyborem w GUI/kodu powyżej)
raw_series = data_btcusdt_frame_daily['close'].values
# raw_series = strategy_equity_frame_daily['equity'].values

## Przygotowanie danych

W materiałach $\lambda$ często jest ustawiona na 1600 dla danych kwartalnych. Natomiast dla dziennych często używa się 14400 lub 100000.

W literaturze (tzw. reguła Ravna-Uhliga), parametr $\lambda$ skaluje się z czwartą potęgą częstotliwości, co by nam dawało:

- kwartalnie: $1600$
- miesięczne: $1600 \times 3^4 = 129\ 600$ (często upraszczane do 144 000)
- dziennie (ok. 90 dni w kwartale): $1600 \times 90^4 \approx \mathbf{100\ 000\ 000} \ (10^8)$

Profesjonalnie dla danych dziennych powinniśmy więc użyć $\lambda=\mathbf{100\ 000\ 000} \ (10^8)$, aby trend był "gładki", a cykle wyraźne.
$10^8$ wygląda na ogromną wartość parametru, dlatego ja ustawiłem $\lambda=\mathbf{6 \ 400 \ 000}$ (standard dla wielu analiz finansowych).

In [53]:
# Przygotowanie danych (Logarytmy + Filtr HP)
print(f"Przetwarzanie serii: {len(raw_series)} punktów")

# Logarytmizacja (stabilizacja wariancji)
data_series = np.log(raw_series)

# Filtracja HP (usunięcie trendu)
# hp_lambda = 1_600
# hp_lambda = 6_400
# hp_lambda = 14_400
# hp_lambda = 100_000
hp_lambda = 6_400_000
cycle, trend = hpfilter(data_series, lamb=hp_lambda)

data_series_centered = cycle  # Składowa cykliczna - scentrowana i stacjonarna

def plot_hp_transformation(raw, log, trend, cycle):
    fig = make_subplots(rows=2, cols=2, 
                         subplot_titles=("1. Surowe dane", "2. Logarytmowane dane",
                                         "3. Trend (HP)", "4. Składowa cykliczna (Stacjonarna)"))
    
    fig.add_trace(go.Scatter(y=raw, name="Raw"), row=1, col=1)
    fig.add_trace(go.Scatter(y=log, name="Log"), row=1, col=2)
    fig.add_trace(go.Scatter(y=trend, name="Trend"), row=2, col=1)
    fig.add_trace(go.Scatter(y=cycle, name="Cycle"), row=2, col=2)
    
    fig.update_layout(title_text="Proces przygotowania danych do analizy spektralnej", height=800, showlegend=False)
    show_plotly(fig)

plot_hp_transformation(raw_series, log_series, trend, cycle)


Przetwarzanie serii: 5607 punktów


### 1. Surowe ceny:
Co widzimy: Wykładniczy wzrost Bitcoina.
Wniosek: Seria jest niestacjonarna (średnia i wariancja zmieniają się w czasie). Wariancja rośnie wraz z ceną. Gdybyśmy zrobili DFT na surowych danych, trend zdominowałby wszystko inne i nie zobaczylibyśmy żadnych cykli.

### 2. Logarytmy cen:
Co widzimy: Cena "wyprostowała się", ale trend nadal idzie w górę.
Wniosek: Zastosowanie log() stabilizuje wariancję (tzw. heteroskedastyczność). Teraz analizujemy procentowe zmiany ceny. To ułatwia algorytmowi DFT znalezienie regularnych wahnięć, które nie zależą od skali ceny.

### 3. Trend (HP):
Co widzimy: Gładką linię pokazującą ogólny kierunek Bitcoina bez "szumu".
Wniosek: Trend w analizie częstotliwościowej to składowa o bardzo niskiej częstotliwości ( idzie w jednym kierunku przez bardzo długi czas ). Aby znaleźć cykle (np. tygodniowe czy miesięczne), musimy tę składową trendu najpierw zidentyfikować i odseparować, bez tego, nasz szereg byłby prawdopodobnie niestacjonarny oraz po lewej stronie ( przy samej krawędzi wykresu ) mielibyśmy ogromny pik wartości zmieniający skalę.

### 4. Składowa cykliczna:
Co widzimy: Szereg oscylujący wokół zera, który wygląda jak "szum z falami".
Wniosek: To jest wynik który chcieliśmy uzyskać. Dzięki odjęciu trendu otrzymaliśmy szereg stacjonarny (ADF Test to potwierdzi). Dopiero na tym wykresie dekompozycja Fouriera ( Punkt 1. projektu ) ma sens. Tutaj ukryte są cykliczne "fale", których szukamy.

Zanim przejdziemy do punktu 1. projektu, sprawdziy czy dane są ju odpowiednio przygotowane.
Dane powinny być stacjonarne, żeby to zastosować zweryfikować przeprowadzimy wsppomniany wcześńiej test ADF, który nam na to odpowiada.
Test ten ma następujące hipotezy:

`H0`: szereg jest niestacjonarny

`H1`: szereg jest stacjonarny

In [54]:
# Weryfikacja stacjonarności (Test ADF) ---
def check_stationarity(timeseries, name: str):
    print(f"-" * 60)
    print(f"Test stacjonarności (ADF) dla: {name}")

    result = adfuller(timeseries, autolag="AIC")
    p_value = result[1]
    print(f"Statystyka ADF: {result[0]:.4f}")
    print(f"p-value: {p_value:.10f}")
    # print("Wartości krytyczne:")
    # for key, value in result[4].items():
    #     print(f"{key}: {value:.4f}")

    if p_value <= 0.05:
        print("p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.")
    else:
        print(
            "p-value > 0.05. Nie ma podstaw do odrzucenia H0. Szereg jest NIESTACJONARNY."
        )


check_stationarity(raw_series, "Cena")
check_stationarity(data_series, "Logarytmy cen (przed filtrem HP)")
check_stationarity(data_series_centered, "Składowa cykliczna (po filtrze HP)")

------------------------------------------------------------
Test stacjonarności (ADF) dla: Cena
Statystyka ADF: -0.9434
p-value: 0.7733470862
p-value > 0.05. Nie ma podstaw do odrzucenia H0. Szereg jest NIESTACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Logarytmy cen (przed filtrem HP)
Statystyka ADF: -3.2092
p-value: 0.0194577359
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.
------------------------------------------------------------
Test stacjonarności (ADF) dla: Składowa cykliczna (po filtrze HP)
Statystyka ADF: -9.8353
p-value: 0.0000000000
p-value <= 0.05. Odrzucamy hipotezę zerową. Szereg jest STACJONARNY.


Zastosowanie testu Augmented Dickey-Fuller (ADF) potwierdziło stacjonarność składowej cyklicznej uzyskanej po filtracji HP ($p-value \approx 0.0$). Wysoka wartość parametru lambda ($6.4 \times 10^6$) pozwoliła na precyzyjne odizolowanie długookresowego trendu przy jednoczesnym zachowaniu istotnych informacji o cyklach rynkowych. Periodogram naiwny wykazuje silną koncentrację mocy w zakresie niskich częstotliwości ($f < 0.05$), co sugeruje dominację cykli trwających powyżej 20 dni. Ze względu na dużą wariancję estymatora naiwnego (charakterystyczny 'las szpilek'), w kolejnym kroku konieczne jest zastosowanie wygładzania spektralnego.

## Wyznaczanie DFT i Periodogramu Naiwnego

### Dyskretna Transformata Fouriera (DFT)
DFT jest podstawowym narzędziem analizy spektralnej, które pozwala przekształcić szereg czasowy z domeny czasu do domeny częstotliwości.

### Periodogram Naiwny
Periodogram naiwny (estymator gęstości widmowej) obliczamy na podstawie kwadratu modułu współczynników DFT:

$$P(f_k) = \frac{1}{N} |X_k|^2$$

Pozwala on zidentyfikować, które częstotliwości "niosą" najwięcej energii (wariancji) w badanym szeregu.

In [55]:
def compute_dft_complex(x):
    """
    Oblicza DFT (Dyskretną Transformatę Fouriera) przy użyciu algorytmu FFT.
    Zwraca współczynniki X_k (liczby zespolone).
    """
    return np.fft.fft(x)

def compute_naive_periodogram(x):
    """
    Oblicza periodogram naiwny na podstawie DFT.
    Zwraca wektor częstotliwości (f) i mocy (P).
    """
    N = len(x)
    X_k = compute_dft_complex(x) # Wywołanie DFT
    
    # Moc: 1/N * |X_k|^2
    power = (1/N) * (np.abs(X_k)**2)
    
    # Częstotliwości (0 do 0.5)
    freqs = np.fft.fftfreq(N, d=1)
    
    # Zwracamy tylko dodatnie częstotliwości
    idx = np.where(freqs >= 0)
    return freqs[idx], power[idx]

# Obliczamy periodogramy dla danych oryginalnych (logarytmy) i scentrowanych (cykl)
f_log, p_log = compute_naive_periodogram(log_series)
f_cyc, p_cyc = compute_naive_periodogram(cycle)

# Tworzymy jeden wspólny wykres porównawczy
fig = go.Figure()

# Dodajemy serię oryginalną (logarytmy)
fig.add_trace(go.Scatter(
    x=f_log[1:], y=p_log[1:], 
    name="Dane oryginalne (Logarytmy)",
    line=dict(color='rgba(100, 100, 100, 0.5)', width=1)
))

# Dodajemy serię scentrowaną (Składowa cykliczna)
fig.add_trace(go.Scatter(
    x=f_cyc[1:], y=p_cyc[1:], 
    name="Dane scentrowane (Składowa cykliczna)",
    line=dict(color='blue', width=1.5)
))

fig.update_layout(
    title="Porównanie periodogramów: Dane oryginalne vs scentrowane",
    xaxis_title="Częstotliwość (f)",
    yaxis_title="Moc (Power)",
    yaxis_type="log",
    hovermode="x unified"
)

show_plotly(fig)

### Porównanie wyników i wnioski:

To co widzimy to są periodogramy naiwne, czyli periodogramy z wykorzystaniem DFT.
Z teorii wynika, że jest on estymatorem niezgodnym. Oznacza to, że wraz z dokładaniem danych, wykres nie staje się gładszy, tylko ma coraz więcej "igieł", które skaczą góra-dół, co też widzimy na wykresach

Głównie interesuje nas to, co się dzieje wokół f=0.
Szary pik ( Dane Originalne ) przy f=0 sięga wartości powyżej 10,000.
Niebieski pik ( Dane scentrowane - detrendowane ) przy f=0 spadł w okolice bliskie zeru.

Spójrzmy jeszcze nadal na lewą część wykresu.
Niebieska linia w tym obszarze jest najwyżej (ma największą moc). To tam ukryte są cykle długookresowe Bitcoina (np. cykl halvingowy czy 4-letni).
Im dalej w prawo (wyższe częstotliwości), tym moc bardziej spada. To oznacza, że "szum dzienny" ma znacznie mniejsze znaczenie niż te powolne wahania.

**Wniosek**: Moc trendu była setki razy większa niż moc cykli. To, że niebieska linia "odkleiła się" od szarej i opadła niżej, to dowód na to, że skutecznie usunęliśmy trend.